In [ ]:
#default_exp discriminators

In [33]:
#export
#hide
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *
from fastai.vision.gan import *
from PIL import Image

import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import utils
from torch.nn import init

from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import pdb
from HiDT.building_blocks import *

# Projection Based Conditional Discriminator

In [56]:
#export
class ProjectionDiscriminator(nn.Module):
    "Projection based discrminator, adapted from: https://github.com/XHChen0528/SNGAN_Projection_Pytorch"
    def __init__(self, num_feat=64, num_classes=0, activation=nn.ReLU()):
        super().__init__()
        self.num_feat = num_feat
        self.num_classes = num_classes
        self.activation = activation
        
        self.blocks = [OptimizedBlock(3, num_feat)]
        self.blocks.extend([
            DisResBlock(num_feat*(2**i), num_feat*(2**(i+1)), downsample=True) for i in range(4)
        ])

        self.l6 = torch.nn.utils.spectral_norm(nn.Linear(num_feat * 16, 1))
        self.style = torch.nn.utils.spectral_norm(
                nn.Linear(3, num_feat * 16))

        self._initialize()

    def _initialize(self):
        init.xavier_uniform_(self.l6.weight.data)
        optional_l_y = getattr(self, 'l_y', None)
        if optional_l_y is not None:
            init.xavier_uniform_(optional_l_y.weight.data)

    def forward(self, x, y=None):
        for block in self.blocks:
            x = block(x)
            
        h = self.activation(x)
        
        h = torch.sum(h, dim=(2, 3))
        
        output = self.l6(h)

        if y is not None:
            output += torch.sum(self.style(y) * h, dim=1, keepdim=True)
            
        return output

In [57]:
inp = torch.randn(4, 3, 256, 256)

In [58]:
dis = ProjectionDiscriminator()

In [59]:
dis(inp).shape

torch.Size([4, 1])

# Unconditional Discriminator

In [16]:
#export
class UnconditionalDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1),
            conv_and_res(64, 128),
            conv_and_res(128, 256),
            conv_and_res(256, 512),
            nn.Conv2d(512, 1, 3, stride=1),
            Flatten(),
            nn.Linear(144, 1)
        )
    
    def forward(self, xb):
        return self.model(xb)

In [17]:
inp = torch.randn(4, 3, 256, 256)

In [18]:
dis = UnconditionalDiscrminator()

In [19]:
dis(inp).shape

torch.Size([4, 1])